In [ ]:
!pip install casadi --quiet
!pip install pymoo --quiet

In [ ]:
import casadi as ca
import numpy as np
from itertools import combinations
from scipy.integrate import solve_ivp
from scipy.linalg import svd
import time
import matplotlib.pyplot as plt
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.core.sampling import Sampling

In [ ]:
# === General Configuration ===
start_time = time.perf_counter()

# === Model configuration for SSF case study ===
nx = 8
nth = 14
Tf = 140
Nt = 101
t_eval = np.linspace(0, Tf, Nt)
param_names = [f"theta{i+1}" for i in range(nth)]

# === Nominal parameter values ===
theta_nom = np.array([
    0.3, 0.05, 5e-3, 0.2, 0.1, 0.0,
    16.0, 3e-3, 3.3, 17.0, 6e-4, 2e3,
    0.17, 0.05
])

sigma_min_threshold = 1e-3
NExp = 50
u0 = 3.146 / 1000
s0 = 1.768685 / 10
x0_val = np.array([0.006878, u0, 0, 0.006878, 0.00885, 0.00467, 0, s0])  # shape (8,)
dx0_val = np.zeros((nx, nth))

# === Model Builder ===
def build_model(free_indices, theta_nom):
    n_free = len(free_indices)
    theta_free = ca.MX.sym("theta", n_free)
    theta_total = [0] * nth
    for i in range(nth):
        if i in free_indices:
            theta_total[i] = theta_free[free_indices.index(i)]
        else:
            theta_total[i] = theta_nom[i]
    mun, kd, ku, mco2, mo2, kp, yxn, kn, yxco2, yxo2, betam, ki, yxs, ms = theta_total
    x = ca.MX.sym("x", nx)
    mu = mun * x[2] / (kn + x[2])
    beta = betam / (1 + ki * x[2])
    xdot = ca.vertcat(
        mu * x[0] - kd * x[0],
        ca.if_else(x[1] > 0, -ku, 0),
        ca.if_else(x[1] > 0, 0.47 * ku - mu * x[0] / yxn, -mu * x[0] / yxn),
        mu * x[0],
        mu * x[0] / yxco2 + mco2 * x[0],
        mu * x[0] / yxo2 + mo2 * x[0],
        beta * x[0] - kp * x[6],
        -mu * x[0] / yxs - ms * x[0]
    )
    x0 = ca.MX(x0_val)
    dx0 = ca.MX(dx0_val[:, :n_free])
    Jx = ca.jacobian(xdot, x)
    Jth = ca.jacobian(xdot, theta_free)
    return {
        'f_rhs': ca.Function("rhs", [x, theta_free], [xdot]),
        'f_Jx': ca.Function("Jx", [x, theta_free], [Jx]),
        'f_Jth': ca.Function("Jth", [x, theta_free], [Jth]),
        'f_x0': lambda _: x0_val,
        'f_dx0': lambda _: dx0_val[:, :n_free],
        'idx_libres': free_indices
    }

# === Simulation with sensitivities ===
def simulate_once(theta_val_free, model):
    x0_val_ = model['f_x0'](theta_val_free)
    dx0_val_ = model['f_dx0'](theta_val_free)
    x_ext = np.concatenate([x0_val_, dx0_val_.flatten()])
    def rhs_meta(t, x_ext):
        x = x_ext[:nx]
        dx_dth = x_ext[nx:].reshape((nx, len(theta_val_free)))
        A = model['f_Jx'](x, theta_val_free).full()
        B = model['f_Jth'](x, theta_val_free).full()
        dx = model['f_rhs'](x, theta_val_free).full().flatten()
        d_dx_dth = A @ dx_dth + B
        return np.concatenate([dx, d_dx_dth.flatten()])
    sol = solve_ivp(rhs_meta, [0, Tf], x_ext, t_eval=t_eval, method='RK45', rtol=1e-4, atol=1e-7)
    if not sol.success:
        raise RuntimeError(" Integration failed")
    x_out = sol.y[:nx].T
    dx_dth_out = sol.y[nx:].T.reshape(Nt, nx, len(theta_val_free))
    return x_out, dx_dth_out

# === Sensitivity matrix ===
observed_indices = [1, 3, 4, 5, 6, 7]
ny = len(observed_indices)

def compute_dydth(x_out, dx_dth_out, theta_val_free):
    dydth = dx_dth_out[:, observed_indices, :].reshape(Nt * ny, len(theta_val_free))
    y_out = x_out[:, observed_indices]
    with np.errstate(divide='ignore', invalid='ignore'):
        dydth_rel = dydth * theta_val_free / np.where(y_out.reshape(-1, 1) != 0, y_out.reshape(-1, 1), 1.0)
    return dydth_rel

# === Evaluation of parameter combinations ===
def evaluate_combination(fixed_idx, theta_nom, NExp=50):
    free_idx = [i for i in range(nth) if i not in fixed_idx]
    model = build_model(free_idx, theta_nom)
    THETA = np.random.uniform(0.5, 1.5, size=(NExp, nth)) * theta_nom
    THETA[0] = theta_nom.copy()
    sigma_min_list = []
    v_last_list = []
    for k in range(NExp):
        theta_k = THETA[k].copy()
        theta_k[fixed_idx] = theta_nom[fixed_idx]
        theta_free = theta_k[free_idx]
        theta_nom_free = theta_nom[free_idx]
        try:
            x_out, dx_dth_out = simulate_once(theta_free, model)
            dydth_rel = compute_dydth(x_out, dx_dth_out, theta_nom_free)
            if np.any(np.isnan(dydth_rel)) or np.any(np.isinf(dydth_rel)):
                raise ValueError("NaNs or Infs in sensitivity matrix")
            _, S, Vh = svd(dydth_rel, full_matrices=False)
            sigma_min_list.append(S[-1])
            v_last_list.append(np.abs(Vh[-1]))
        except Exception as e:
            print(f" Error for combination {fixed_idx}: {e}")
            continue
    if len(sigma_min_list) == 0:
        return None
    sigma_min = np.min(sigma_min_list)
    v_last_array = np.array(v_last_list)
    mean_v = np.mean(v_last_array, axis=0)
    std_v = np.std(v_last_array, axis=0)
    print(f"🔍 Fix {[param_names[i] for i in fixed_idx]} → σ_min = {sigma_min:.2e}")
    return {
        'fixed': [param_names[i] for i in fixed_idx],
        'free': [param_names[i] for i in free_idx],
        'sigma_min': sigma_min,
        'mean_v': mean_v,
        'std_v': std_v
    }

# === Loop through all combinations ===
results = []

for r in range(nth + 1):
    candidates_r = []
    for comb in combinations(range(nth), r):
        res = evaluate_combination(list(comb), theta_nom, NExp=NExp)
        if res is not None:
            candidates_r.append(res)
    if any(r['sigma_min'] >= sigma_min_threshold for r in candidates_r):
        results.extend(candidates_r)
        print(f"\n Analysis stopped at r = {r} fixed parameters.")
        break
    results.extend(candidates_r)

# === Report best results ===
valid_results = [r for r in results if r['sigma_min'] >= sigma_min_threshold]
if valid_results:
    min_fixed = min(len(r['fixed']) for r in valid_results)
    best_results = [r for r in valid_results if len(r['fixed']) == min_fixed]
    print(f"\n Optimal combinations (σ_min ≥ {sigma_min_threshold:.0e}):")
    for r in best_results:
        print(f" Fix {r['fixed']} → σ_min = {r['sigma_min']:.2e}")
else:
    print(f"\n No valid combination found with σ_min ≥ {sigma_min_threshold:.0e}")

elapsed_time = time.perf_counter() - start_time
print(f"\nSimulation time: {elapsed_time:.3e} s.")

🔍 Fix [] → σ_min = 3.83e-18
🔍 Fix ['theta1'] → σ_min = 3.00e-19
🔍 Fix ['theta2'] → σ_min = 5.37e-18
🔍 Fix ['theta3'] → σ_min = 3.08e-19
🔍 Fix ['theta4'] → σ_min = 1.15e-17
🔍 Fix ['theta5'] → σ_min = 3.27e-18
🔍 Fix ['theta6'] → σ_min = 1.50e-03
🔍 Fix ['theta7'] → σ_min = 5.33e-18
🔍 Fix ['theta8'] → σ_min = 1.77e-17
🔍 Fix ['theta9'] → σ_min = 2.62e-19
🔍 Fix ['theta10'] → σ_min = 2.46e-18
🔍 Fix ['theta11'] → σ_min = 1.04e-17
🔍 Fix ['theta12'] → σ_min = 3.25e-20
🔍 Fix ['theta13'] → σ_min = 3.77e-18
🔍 Fix ['theta14'] → σ_min = 1.01e-17

 Analysis stopped at r = 1 fixed parameters.

 Optimal combinations (σ_min ≥ 1e-03):
 Fix ['theta6'] → σ_min = 1.50e-03

Simulation time: 3.543e+01 s.


In [ ]:
# === General Configuration ===
# === Model configuration for SSF case study ===
nx = 8
nth = 14
Tf = 140
Nt = 101
t_eval = np.linspace(0, Tf, Nt)
param_names = [f"theta{i+1}" for i in range(nth)]

# === Nominal parameter values ===
theta_nom = np.array([
    0.3, 0.05, 5e-3, 0.2, 0.1, 0.0,
    16.0, 3e-3, 3.3, 17.0, 6e-4, 2e3,
    0.17, 0.05
])

sigma_min_threshold = 1e-3
NExp = 50

u0 = 3.146 / 1000
s0 = 1.768685 / 10
x0_val = np.array([0.006878, u0, 0, 0.006878, 0.00885, 0.00467, 0, s0])  # shape (8,)
dx0_val = np.zeros((nx, nth))

# === Model Constructor ===
def construct_model(free_idx, theta_nom):
    n_free = len(free_idx)
    theta_free = ca.MX.sym("theta", n_free)
    theta_total = [0]*nth
    for i in range(nth):
        if i in free_idx:
            theta_total[i] = theta_free[free_idx.index(i)]
        else:
            theta_total[i] = theta_nom[i]
    mun, kd, ku, mco2, mo2, kp, yxn, kn, yxco2, yxo2, betam, ki, yxs, ms = theta_total
    x = ca.MX.sym("x", nx)
    mu = mun * x[2] / (kn + x[2])
    beta = betam / (1 + ki * x[2])
    xdot = ca.vertcat(
        mu * x[0] - kd * x[0],
        ca.if_else(x[1] > 0, -ku, 0),
        ca.if_else(x[1] > 0, 0.47 * ku - mu * x[0] / yxn, -mu * x[0] / yxn),
        mu * x[0],
        mu * x[0] / yxco2 + mco2 * x[0],
        mu * x[0] / yxo2 + mo2 * x[0],
        beta * x[0] - kp * x[6],
        -mu * x[0] / yxs - ms * x[0]
    )
    x0_val = np.array([0.006878, 3.146 / 1000, 0, 0.006878, 0.00885, 0.00467, 0, 1.768685 / 10])
    dx0_val = np.zeros((nx, nth))
    dx0 = ca.MX(dx0_val[:, :n_free])
    Jx = ca.jacobian(xdot, x)
    Jth = ca.jacobian(xdot, theta_free)
    return {
        'f_rhs': ca.Function("rhs", [x, theta_free], [xdot]),
        'f_Jx': ca.Function("Jx", [x, theta_free], [Jx]),
        'f_Jth': ca.Function("Jth", [x, theta_free], [Jth]),
        'f_x0': lambda _: x0_val,
        'f_dx0': lambda _: dx0_val[:, :n_free],
        'idx_libres': free_idx
    }

# === Simulation with sensitivities ===
def simulate_once(theta_val_free, model):
    x0_val_ = model['f_x0'](theta_val_free)
    dx0_val_ = model['f_dx0'](theta_val_free)
    x_ext = np.concatenate([x0_val_, dx0_val_.flatten()])
    def rhs_meta(t, x_ext):
        x = x_ext[:nx]
        dx_dth = x_ext[nx:].reshape((nx, len(theta_val_free)))
        A = model['f_Jx'](x, theta_val_free).full()
        B = model['f_Jth'](x, theta_val_free).full()
        dx = model['f_rhs'](x, theta_val_free).full().flatten()
        d_dx_dth = A @ dx_dth + B
        return np.concatenate([dx, d_dx_dth.flatten()])
    sol = solve_ivp(rhs_meta, [0, Tf], x_ext, t_eval=t_eval, method='RK45', rtol=1e-4, atol=1e-7)
    if not sol.success:
        raise RuntimeError(" Integration failed")
    x_out = sol.y[:nx].T
    dx_dth_out = sol.y[nx:].T.reshape(Nt, nx, len(theta_val_free))
    return x_out, dx_dth_out

# === Sensitivity matrix ===
observed_indices = [1, 3, 4, 5, 6, 7]
ny = len(observed_indices)

def compute_dydth(x_out, dx_dth_out, theta_val_free):
    dydth = dx_dth_out[:, observed_indices, :].reshape(Nt * ny, len(theta_val_free))
    y_out = x_out[:, observed_indices]
    with np.errstate(divide='ignore', invalid='ignore'):
        dydth_rel = dydth * theta_val_free / np.where(y_out.reshape(-1, 1) != 0, y_out.reshape(-1, 1), 1.0)
    return dydth_rel

# === Evaluation of parameter combinations ===
def evaluate_combination(fixed_idx, theta_nom, NExp=50):
    free_idx = [i for i in range(nth) if i not in fixed_idx]
    model = construct_model(free_idx, theta_nom)
    THETA = np.random.uniform(0.5, 1.5, size=(NExp, nth)) * theta_nom
    THETA[0] = theta_nom.copy()
    sigma_min_list = []
    v_last_list = []
    for k in range(NExp):
        theta_k = THETA[k].copy()
        theta_k[fixed_idx] = theta_nom[fixed_idx]
        theta_free = theta_k[free_idx]
        theta_nom_free = theta_nom[free_idx]
        try:
            x_out, dx_dth_out = simulate_once(theta_free, model)
            dydth_rel = compute_dydth(x_out, dx_dth_out, theta_nom_free)
            if np.any(np.isnan(dydth_rel)) or np.any(np.isinf(dydth_rel)):
                raise ValueError("NaNs or Infs in sensitivity matrix")
            _, S, Vh = svd(dydth_rel, full_matrices=False)
            sigma_min_list.append(S[-1])
            v_last_list.append(np.abs(Vh[-1]))
        except Exception as e:
            print(f" Error for combination {fixed_idx}: {e}")
            continue
    if len(sigma_min_list) == 0:
        return None
    sigma_min = np.min(sigma_min_list)
    v_last_array = np.array(v_last_list)
    mean_v = np.mean(v_last_array, axis=0)
    std_v = np.std(v_last_array, axis=0)
    print(f" Fix {[param_names[i] for i in fixed_idx]} → σ_min = {sigma_min:.2e}")
    return {
        'fixed': [param_names[i] for i in fixed_idx],
        'free': [param_names[i] for i in free_idx],
        'fixed_idx': fixed_idx,
        'sigma_min': sigma_min,
        'mean_v': mean_v,
        'std_v': std_v
    }

# === Hierarchical search based on v_last priority ===
def hierarchical_search_vlast(theta_nom, evaluate_combination, sigma_min_threshold=1e-3, NExp=50):
    Nth = len(theta_nom)
    print(" Initial analysis with all parameters free...")
    res0 = evaluate_combination([], theta_nom, NExp)
    if res0 is None:
        raise RuntimeError("Initial analysis with all parameters free failed.")
    v_base = np.abs(res0['mean_v'])
    priority_order = list(np.argsort(-v_base))
    contribution_threshold = 1e-2
    priority_order = [i for i in priority_order if v_base[i] > contribution_threshold]
    history = set()
    results = []
    for num_fixed in range(1, Nth + 1):
        new_combinations = []
        if num_fixed == 1:
            for i in priority_order:
                new_combinations.append((i,))
        else:
            previous = [tuple(sorted(res['fixed_idx'])) for res in results if len(res['fixed_idx']) == num_fixed - 1]
            for comb in previous:
                remaining = [i for i in priority_order if i not in comb]
                for new_param in remaining:
                    new_comb = tuple(sorted(comb + (new_param,)))
                    new_combinations.append(new_comb)
        new_combinations = [c for c in new_combinations if c not in history]
        if not new_combinations:
            break
        for comb in new_combinations:
            res = evaluate_combination(list(comb), theta_nom, NExp)
            if res is not None:
                res['fixed_idx'] = list(comb)
                results.append(res)
                if res['sigma_min'] >= sigma_min_threshold:
                    print(f"\n Threshold reached by fixing {res['fixed']} → σ_min = {res['sigma_min']:.2e}")
                    return results
            history.add(comb)
    print("\n No combination reached the σ_min threshold.")
    return results

# === Execute hierarchical search ===
start_time = time.perf_counter()
results = hierarchical_search_vlast(theta_nom, evaluate_combination)
elapsed_time = time.perf_counter() - start_time
print(f"\nSimulation time: {elapsed_time:.3e} s.")

 Initial analysis with all parameters free...
 Fix [] → σ_min = 1.47e-18
 Fix ['theta6'] → σ_min = 1.68e-03

 Threshold reached by fixing ['theta6'] → σ_min = 1.68e-03

Simulation time: 4.084e+00 s.


In [ ]:
import casadi as ca
import numpy as np
from scipy.integrate import solve_ivp
from scipy.linalg import svd
import time

# === Configuration ===
nx = 8
nth = 14
Tf = 140
Nt = 101
t_eval = np.linspace(0, Tf, Nt)
param_names = [f"theta{i+1}" for i in range(nth)]

theta_nom = np.array([
    0.3, 0.05, 5e-3, 0.2, 0.1, 0.0,
    16.0, 3e-3, 3.3, 17.0, 6e-4, 2e3,
    0.17, 0.05
])

sigma_min_threshold = 1e-3
NExp = 50

def construct_model(free_idx, theta_nom):
    n_free = len(free_idx)
    theta_free = ca.MX.sym("theta", n_free)
    theta_total = [0] * nth
    for i in range(nth):
        if i in free_idx:
            theta_total[i] = theta_free[free_idx.index(i)]
        else:
            theta_total[i] = theta_nom[i]
    mun, kd, ku, mco2, mo2, kp, yxn, kn, yxco2, yxo2, betam, ki, yxs, ms = theta_total
    x = ca.MX.sym("x", nx)
    mu = mun * x[2] / (kn + x[2])
    beta = betam / (1 + ki * x[2])
    xdot = ca.vertcat(
        mu * x[0] - kd * x[0],
        ca.if_else(x[1] > 0, -ku, 0),
        ca.if_else(x[1] > 0, 0.47 * ku - mu * x[0] / yxn, -mu * x[0] / yxn),
        mu * x[0],
        mu * x[0] / yxco2 + mco2 * x[0],
        mu * x[0] / yxo2 + mo2 * x[0],
        beta * x[0] - kp * x[6],
        -mu * x[0] / yxs - ms * x[0]
    )
    x0_val = np.array([0.006878, 3.146 / 1000, 0, 0.006878, 0.00885, 0.00467, 0, 1.768685 / 10])
    dx0_val = np.zeros((nx, nth))
    dx0 = ca.MX(dx0_val[:, :n_free])
    Jx = ca.jacobian(xdot, x)
    Jth = ca.jacobian(xdot, theta_free)
    return {
        'f_rhs': ca.Function("rhs", [x, theta_free], [xdot]),
        'f_Jx': ca.Function("Jx", [x, theta_free], [Jx]),
        'f_Jth': ca.Function("Jth", [x, theta_free], [Jth]),
        'f_x0': lambda _: x0_val,
        'f_dx0': lambda _: dx0_val[:, :n_free],
        'free_idx': free_idx
    }

# Simulate once for a set of free parameters
def simulate_once(theta_val_free, model):
    x0_val_ = model['f_x0'](theta_val_free)
    dx0_val_ = model['f_dx0'](theta_val_free)
    x_ext = np.concatenate([x0_val_, dx0_val_.flatten()])
    def rhs_meta(t, x_ext):
        x = x_ext[:nx]
        dx_dth = x_ext[nx:].reshape((nx, len(theta_val_free)))
        A = model['f_Jx'](x, theta_val_free).full()
        B = model['f_Jth'](x, theta_val_free).full()
        dx = model['f_rhs'](x, theta_val_free).full().flatten()
        d_dx_dth = A @ dx_dth + B
        return np.concatenate([dx, d_dx_dth.flatten()])
    sol = solve_ivp(rhs_meta, [0, Tf], x_ext, t_eval=t_eval, method='RK45', rtol=1e-4, atol=1e-7)
    if not sol.success:
        raise RuntimeError("Integration failed")
    x_out = sol.y[:nx].T
    dx_dth_out = sol.y[nx:].T.reshape(Nt, nx, len(theta_val_free))
    return x_out, dx_dth_out

# Compute relative sensitivities
def compute_dydth(x_out, dx_dth_out, theta_val_free):
    observed_indices = [1, 3, 4, 5, 6, 7]
    ny = len(observed_indices)
    dydth = dx_dth_out[:, observed_indices, :].reshape(Nt * ny, len(theta_val_free))
    y_out = x_out[:, observed_indices]
    with np.errstate(divide='ignore', invalid='ignore'):
        dydth_rel = dydth * theta_val_free / np.where(y_out.reshape(-1, 1) != 0, y_out.reshape(-1, 1), 1.0)
    return dydth_rel

# Evaluate a given fixed set, return SVD metrics
def evaluate_combination(fixed_idx, theta_nom, NExp=50):
    free_idx = [i for i in range(nth) if i not in fixed_idx]
    if not free_idx:
        print(f"⛔️ All parameters fixed. Discarded.")
        return None
    model = construct_model(free_idx, theta_nom)
    THETA = np.random.uniform(0.5, 1.5, size=(NExp, nth)) * theta_nom
    THETA[0] = theta_nom.copy()
    sigma_min_list = []
    v_last_list = []
    for k in range(NExp):
        theta_k = THETA[k].copy()
        theta_k[fixed_idx] = theta_nom[fixed_idx]
        theta_free = theta_k[free_idx]
        theta_nom_free = theta_nom[free_idx]
        try:
            x_out, dx_dth_out = simulate_once(theta_free, model)
            dydth_rel = compute_dydth(x_out, dx_dth_out, theta_nom_free)
            if np.any(np.isnan(dydth_rel)) or np.any(np.isinf(dydth_rel)):
                continue
            _, S, Vh = svd(dydth_rel, full_matrices=False)
            sigma_min_list.append(S[-1])
            v_last_list.append(np.abs(Vh[-1]))
        except Exception as e:
            continue
    if len(sigma_min_list) == 0:
        return None
    sigma_min = np.min(sigma_min_list)
    v_last_array = np.array(v_last_list)
    mean_v = np.mean(v_last_array, axis=0)
    std_v = np.std(v_last_array, axis=0)
    print(f"Fix {[param_names[i] for i in fixed_idx]} → σ_min = {sigma_min:.2e}")
    return {
        'fixed': [param_names[i] for i in fixed_idx],
        'free': [param_names[i] for i in free_idx],
        'sigma_min': sigma_min,
        'mean_v': mean_v,
        'std_v': std_v,
        'fixed_idx': fixed_idx,
        'free_idx': free_idx
    }

# Greedy hierarchical search
def hierarchical_search(theta_nom, evaluate_combination, sigma_min_threshold=1e-3, NExp=50):
    Nth = len(theta_nom)
    fixed_idx = []
    results = []
    for _ in range(Nth):
        res = evaluate_combination(fixed_idx, theta_nom, NExp)
        if res is None:
            print("All parameters fixed, exiting.")
            break
        if res["sigma_min"] >= sigma_min_threshold:
            print(f"Threshold reached: fix {res['fixed']} → σ_min = {res['sigma_min']:.2e}")
            results.append(res)
            break
        # Choose the parameter with largest |mean v_last| to fix next
        v_last = np.abs(res["mean_v"])
        # Ignore already fixed
        v_last_masked = np.zeros_like(v_last)
        for i, idx in enumerate(res["free_idx"]):
            v_last_masked[i] = v_last[i]
        idx_to_fix = res["free_idx"][np.argmax(v_last_masked)]
        fixed_idx.append(idx_to_fix)
        results.append(res)
        print(f"Adding parameter {param_names[idx_to_fix]} to fixed set.")
    return results

# Run the search
start_time = time.perf_counter()
results = hierarchical_search(theta_nom, evaluate_combination, sigma_min_threshold, NExp)
elapsed_time = time.perf_counter() - start_time
print(f"\nTotal search time: {elapsed_time:.3f} seconds.")

# Print the best combination
if results:
    best = results[-1]
    print(f"\nBest combination (σ_min ≥ {sigma_min_threshold}):")
    print(f"Fixed: {best['fixed']}")
    print(f"σ_min: {best['sigma_min']:.2e}")
else:
    print("No valid combination found.")

Fix [] → σ_min = 2.55e-19
Adding parameter theta6 to fixed set.
Fix ['theta6'] → σ_min = 1.01e-03
Threshold reached: fix ['theta6'] → σ_min = 1.01e-03

Total search time: 5.071 seconds.

Best combination (σ_min ≥ 0.001):
Fixed: ['theta6']
σ_min: 1.01e-03
